-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Stored Views

In this notebook, we'll give a quick overview of how stored views are created and managed. Recall that stored views differ from DataFrames and temp views by persisting to a database (allowing other users to leverage pre-defined logic to materialize results). Views register the logic required to calculate a result (which will be evaluated when the query is executed). Views defined against Delta Lake sources are guaranteed to always query the latest version of each data source.

The goal of this notebook is to generate a view that allows the analysts from our partner gyms to examine how use of Moovio devices and workouts impact gym activity.

<img src="https://files.training.databricks.com/images/ade/ADE_arch_gym_report.png" width="60%" />

## Learning Objectives
By the end of this lesson, students will be able to:
- Display the query plan associated with a view
- Describe how results are materialized from Delta Lake tables

## Setup

In [0]:
%run ../Includes/Classroom-Setup-5.1

Creating the database "dbacademy_odl_user_771624_databrickslabs_com_adewd_5_1"
Creating bronze...(4 seconds / 10,841,978 records)
Creating user_lookup...(3 seconds / 100 records)
Creating gym_mac_logs...(3 seconds / 314 records)

Processing the workouts_silver table...(14 seconds / 1,514 records)
Processing the completed_workouts table...(3 seconds / 757 records)

Predefined Paths:
 DA.paths.working_dir: dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/5.1
 DA.paths.user_db: dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/5.1/5_1.db
 DA.paths.checkpoints: dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/5.1/_checkpoints

Predefined tables in dbacademy_odl_user_771624_databrickslabs_com_adewd_5_1:
 bronze
 completed_workouts
 gym_mac_logs
 user_lookup
 workouts_silver

Validating datasets...(1 seconds)

Setup completed in 29 seconds

## Examine Gym Logs

Start by reviewing the schema for your gym logs.

In [0]:
gymDF = spark.table("gym_mac_logs")
gymDF.printSchema()

root
-- first_timestamp: double (nullable = true)
-- gym: long (nullable = true)
-- last_timestamp: double (nullable = true)
-- mac: string (nullable = true)

A Spark DataFrame and a view are nearly identical constructs. By calling **`explain`** on our DataFrame, we can see that our source table is a set of instructions to deserialize the files containing our data.

In [0]:
gymDF.explain("formatted")

== Physical Plan ==
* ColumnarToRow (2)
+- Scan parquet dbacademy_odl_user_771624_databrickslabs_com_adewd_5_1.gym_mac_logs (1)


(1) Scan parquet dbacademy_odl_user_771624_databrickslabs_com_adewd_5_1.gym_mac_logs
Output [4]: [first_timestamp#40691, gym#40692L, last_timestamp#40693, mac#40694]
Batched: true
Location: PreparedDeltaFileIndex [dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/5.1/5_1.db/gym_mac_logs]
ReadSchema: struct<first_timestamp:double,gym:bigint,last_timestamp:double,mac:string>

(2) ColumnarToRow [codegen id : 1]
Input [4]: [first_timestamp#40691, gym#40692L, last_timestamp#40693, mac#40694]

## Examine Workout Data

Rather than trying to capture every possible metric in our view, we'll create a summary of values that might be of interest to our gym analysts.

The data we're receiving from our gyms indicates the first and last timestamp recorded for user devices, indicated by mac address.

In [0]:
%sql
SELECT * FROM gym_mac_logs

first_timestamp,gym,last_timestamp,mac
1.575202519E9,1,1.575208579E9,3c:97:06:9e:63:9c
1.575194342E9,2,1.575200195E9,e5:a8:d5:73:cf:33
1.575220992E9,2,1.575225715E9,e4:2c:d4:ea:e4:b4
1.575184296E9,2,1.575191636E9,54:bf:e4:40:52:b9
1.575221256E9,3,1.575229065E9,c3:3e:16:08:f9:47
1.575221584E9,3,1.57522577E9,4c:53:51:88:59:ab
1.575184445E9,4,1.57518829E9,dd:45:d2:37:a8:0e
1.575221226E9,4,1.575228054E9,df:f9:dc:5e:e2:a8
1.575202031E9,5,1.57520663E9,a4:eb:49:d9:9b:1d
1.57519375E9,5,1.575196735E9,00:6c:6c:53:51:ef


## Construct a Query

Our **`completed_workouts`** table indicates start and stop time for each user workout.

Use the cell below to construct a query that identifies:
- Each date a user completed at least one workout
- The earliest **`start_time`** for any workout each day
- The latest **`end_time`** for any workout each day
- The list of all workouts completed by a user each day

In [0]:
%sql
SELECT user_id, to_date(start_time) date, collect_set(workout_id), min(start_time), max(end_time)
FROM completed_workouts
GROUP BY user_id, to_date(start_time)
/*SELECT * 
FROM completed_workouts
LIMIT 10*/

user_id,date,collect_set(workout_id),min(start_time),max(end_time)
11745,2019-12-10,List(38),2019-12-10T07:25:52.000+0000,2019-12-10T08:21:20.000+0000
11745,2019-12-13,List(22),2019-12-13T13:52:00.000+0000,2019-12-13T15:00:16.000+0000
12140,2019-12-05,"List(9, 5)",2019-12-05T07:51:28.000+0000,2019-12-05T09:16:48.000+0000
12140,2019-12-12,List(5),2019-12-12T07:15:12.000+0000,2019-12-12T07:49:20.000+0000
12227,2019-12-11,"List(0, 32)",2019-12-11T17:16:48.000+0000,2019-12-11T18:48:32.000+0000
12227,2019-12-13,List(13),2019-12-13T12:28:48.000+0000,2019-12-13T13:24:16.000+0000
12227,2019-12-15,"List(49, 4)",2019-12-15T07:30:08.000+0000,2019-12-15T08:10:40.000+0000
12474,2019-12-01,"List(0, 35)",2019-12-01T17:27:28.000+0000,2019-12-01T18:44:16.000+0000
12474,2019-12-02,List(45),2019-12-02T07:38:40.000+0000,2019-12-02T08:17:04.000+0000
12474,2019-12-03,"List(39, 44)",2019-12-03T07:21:36.000+0000,2019-12-03T08:14:56.000+0000


## Expand on the Query

Now we'll join this data back to the MAC logs sent by the gym to create our view.

We'll retain the **`mac_address`** as our identifier, which we can grab from the **`user_lookup`** table.

We'll also add columns to calculate the total number of minutes elapsed during a user's visit to the gym, as well as the total number of minutes elapsed between the beginning of their first workout and the end of their final workout.

In [0]:
%sql
SELECT gym, mac_address, date, workouts, (last_timestamp - first_timestamp)/60 minutes_in_gym, (to_unix_timestamp(end_workout) - to_unix_timestamp(start_workout))/60 minutes_exercising
FROM gym_mac_logs c
INNER JOIN (
  SELECT b.mac_address, to_date(start_time) date, collect_set(workout_id) workouts, min(start_time) start_workout, max(end_time) end_workout
      FROM completed_workouts a
      INNER JOIN user_lookup b
      ON a.user_id = b.user_id
      GROUP BY mac_address, to_date(start_time)
  ) d
  ON c.mac = d.mac_address AND to_date(CAST(c.first_timestamp AS timestamp)) = d.date

gym,mac_address,date,workouts,minutes_in_gym,minutes_exercising
5,00:6c:6c:53:51:ef,2019-12-01,"List(49, 43)",49.75,40.53333333333333
5,00:6c:6c:53:51:ef,2019-12-02,"List(21, 22)",97.66666666666667,91.73333333333333
5,00:6c:6c:53:51:ef,2019-12-03,"List(24, 21)",47.1,38.4
5,00:6c:6c:53:51:ef,2019-12-04,List(27),57.46666666666667,46.93333333333333
5,00:6c:6c:53:51:ef,2019-12-05,"List(31, 21)",84.91666666666667,68.26666666666667
5,00:6c:6c:53:51:ef,2019-12-07,"List(2, 7)",94.45,78.93333333333334
5,00:6c:6c:53:51:ef,2019-12-08,"List(39, 41)",56.63333333333333,46.93333333333333
5,00:6c:6c:53:51:ef,2019-12-09,"List(38, 36)",125.31666666666666,110.93333333333334
5,00:6c:6c:53:51:ef,2019-12-10,List(23),69.05,66.13333333333334
5,00:6c:6c:53:51:ef,2019-12-11,List(9),60.43333333333333,42.666666666666664


## Register View with Final Logic

Create a (non-temporary) view called **`gym_user_stats`** using the query above.

**`CREATE VIEW IF NOT EXISTS gym_user_stats AS (...)`**

In [0]:
%sql
-- TODO
CREATE VIEW IF NOT EXISTS gym_user_stats AS
(
SELECT gym, mac_address, date, workouts, (last_timestamp - first_timestamp)/60 minutes_in_gym, (to_unix_timestamp(end_workout) - to_unix_timestamp(start_workout))/60 minutes_exercising
FROM gym_mac_logs c
INNER JOIN (
  SELECT b.mac_address, to_date(start_time) date, collect_set(workout_id) workouts, min(start_time) start_workout, max(end_time) end_workout
      FROM completed_workouts a
      INNER JOIN user_lookup b
      ON a.user_id = b.user_id
      GROUP BY mac_address, to_date(start_time)
  ) d
  ON c.mac = d.mac_address AND to_date(CAST(c.first_timestamp AS timestamp)) = d.date)

In [0]:
# Check your work
assert spark.sql("SHOW TABLES").filter("tableName='gym_user_stats'").count() >= 1, "View 'gym_user_stats' does not exist."
assert spark.sql("SHOW TABLES").filter("tableName='gym_user_stats'").first()["isTemporary"]==False, "View 'gym_user_stats' should be not temporary."
assert spark.sql("DESCRIBE EXTENDED gym_user_stats").filter("col_name='Type'").first()['data_type']=='VIEW', "Found a table 'gym_user_stats' when a view was expected."
assert spark.table("gym_user_stats").count() == 304, "Incorrect query used for view 'gym_user_stats'."
print("All tests passed.")

All tests passed.

We can see that our view is simply storing the Spark plan for our query.

In [0]:
spark.table("gym_user_stats").explain("formatted")

== Physical Plan ==
AdaptiveSparkPlan (16)
+- Project (15)
 +- BroadcastHashJoin Inner BuildLeft (14)
 :- BroadcastExchange (3)
 : +- Filter (2)
 : +- Scan parquet dbacademy_odl_user_771624_databrickslabs_com_adewd_5_1.gym_mac_logs (1)
 +- ObjectHashAggregate (13)
 +- Exchange (12)
 +- ObjectHashAggregate (11)
 +- Project (10)
 +- BroadcastHashJoin Inner BuildRight (9)
 :- Filter (5)
 : +- Scan parquet dbacademy_odl_user_771624_databrickslabs_com_adewd_5_1.completed_workouts (4)
 +- BroadcastExchange (8)
 +- Filter (7)
 +- Scan parquet dbacademy_odl_user_771624_databrickslabs_com_adewd_5_1.user_lookup (6)


(1) Scan parquet dbacademy_odl_user_771624_databrickslabs_com_adewd_5_1.gym_mac_logs
Output [4]: [first_timestamp#41309, gym#41310L, last_timestamp#41311, mac#41312]
Batched: true
Location: PreparedDeltaFileIndex [dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/5.1/5_1.db/gym_mac_logs]
PushedFilters: [IsNotNull(mac), IsNotNull(first_timestamp)]
ReadSchema: struct<first_timestamp:double,gym:bigint,last_timestamp:double,mac:string>

(2) Filter
Input [4]: [first_timestamp#41309, gym#41310L, last_timestamp#41311, mac#41312]
Condition : (isnotnull(mac#41312) AND isnotnull(first_timestamp#41309))

(3) BroadcastExchange
Input [4]: [first_timestamp#41309, gym#41310L, last_timestamp#41311, mac#41312]
Arguments: HashedRelationBroadcastMode(ArrayBuffer(input[3, string, false], cast(cast(input[0, double, false] as timestamp) as date)),false), [id=#30793]

(4) Scan parquet dbacademy_odl_user_771624_databrickslabs_com_adewd_5_1.completed_workouts
Output [4]: [user_id#41313, workout_id#41314, start_time#41316, end_time#41317]
Batched: true
Location: PreparedDeltaFileIndex [dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/5.1/5_1.db/completed_workouts]
PushedFilters: [IsNotNull(user_id)]
ReadSchema: struct<user_id:int,workout_id:int,start_time:timestamp,end_time:timestamp>

(5) Filter
Input [4]: [user_id#41313, workout_id#41314, start_time#41316, end_time#41317]
Condition : (isnotnull(user_id#41313) AND isnotnull(cast(start_time#41316 as date)))

(6) Scan parquet dbacademy_odl_user_771624_databrickslabs_com_adewd_5_1.user_lookup
Output [2]: [mac_address#41321, user_id#41322L]
Batched: true
Location: PreparedDeltaFileIndex [dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/5.1/5_1.db/user_lookup]
PushedFilters: [IsNotNull(user_id), IsNotNull(mac_address)]
ReadSchema: struct<mac_address:string,user_id:bigint>

(7) Filter
Input [2]: [mac_address#41321, user_id#41322L]
Condition : (isnotnull(user_id#41322L) AND isnotnull(mac_address#41321))

(8) BroadcastExchange
Input [2]: [mac_address#41321, user_id#41322L]
Arguments: HashedRelationBroadcastMode(List(input[1, bigint, false]),false), [id=#30785]

(9) BroadcastHashJoin
Left keys [1]: [cast(user_id#41313 as bigint)]
Right keys [1]: [user_id#41322L]
Join condition: None

(10) Project
Output [5]: [workout_id#41314, start_time#41316, end_time#41317, mac_address#41321, cast(start_time#41316 as date) AS _groupingexpression#41335]
Input [6]: [user_id#41313, workout_id#41314, start_time#41316, end_time#41317, mac_address#41321, user_id#41322L]

(11) ObjectHashAggregate
Input [5]: [workout_id#41314, start_time#41316, end_time#41317, mac_address#41321, _groupingexpression#41335]
Keys [2]: [mac_address#41321, _groupingexpression#41335]
Functions [3]: [partial_collect_set(workout_id#41314, 0, 0) AS buf#41337, partial_min(start_time#41316) AS min#41339, partial_max(end_time#41317) AS max#41341]
Aggregate Attributes [3]: [buf#41336, min#41338, max#41340]
Results [5]: [mac_address#41321, _groupingexpression#41335, buf#41337, min#41339, max#41341]

(12) Exchange
Input [5]: [mac_address#41321, _groupingexpression#41335, buf#41337, min#41339, max#41341]
Arguments: hashpartitioning(mac_address#41321, _groupingexpression#41335, 4), ENSURE_REQUIREMENTS, [id=#30790]

(13) ObjectHashAggregate
Input [5]: [mac_address#41321, _groupingexpression#41335, buf#41337, min#41339, max#41

When we execute a query against this view, we will process the plan to generate the logically correct result.

Note that while the data may end up in the Delta Cache, this result is not guaranteed to be persisted, and is only cached for the currently active cluster.

In [0]:
%sql
SELECT *
FROM gym_user_stats
WHERE gym = 5

gym,mac_address,date,workouts,minutes_in_gym,minutes_exercising
5,00:6c:6c:53:51:ef,2019-12-01,"List(49, 43)",49.75,40.53333333333333
5,00:6c:6c:53:51:ef,2019-12-02,"List(21, 22)",97.66666666666667,91.73333333333333
5,00:6c:6c:53:51:ef,2019-12-03,"List(24, 21)",47.1,38.4
5,00:6c:6c:53:51:ef,2019-12-04,List(27),57.46666666666667,46.93333333333333
5,00:6c:6c:53:51:ef,2019-12-05,"List(31, 21)",84.91666666666667,68.26666666666667
5,00:6c:6c:53:51:ef,2019-12-07,"List(2, 7)",94.45,78.93333333333334
5,00:6c:6c:53:51:ef,2019-12-08,"List(39, 41)",56.63333333333333,46.93333333333333
5,00:6c:6c:53:51:ef,2019-12-09,"List(38, 36)",125.31666666666666,110.93333333333334
5,00:6c:6c:53:51:ef,2019-12-10,List(23),69.05,66.13333333333334
5,00:6c:6c:53:51:ef,2019-12-11,List(9),60.43333333333333,42.666666666666664


## An Aside on ACLs

While Databricks has extensive support for ACLs, by default these are not enforced for standard data engineering clusters. As such, the default permissions for this view are set to all users, and no owner has been declared.

In [0]:
import py4j

try:
    spark.sql("SHOW GRANT ON VIEW gym_user_stats")
    
except py4j.protocol.Py4JJavaError as e:
    print("Error: " + e.java_exception.getMessage())
    print("Solution: Consider enabling Table Access Control to demonstrate this feature.")

Error: Trying to perform permission action on Hive Metastore /CATALOG/`hive_metastore`/DATABASE/`dbacademy_odl_user_771624_databrickslabs_com_adewd_5_1`/TABLE/`gym_user_stats` but Table Access Control is not enabled on this cluster. 
Solution: Consider enabling Table Access Control to demonstrate this feature.

While the privileges for this view may not be especially sensitive, we can see that our bronze table (which contains ALL our raw data) is also currently stored in this fashion.

Again, ACLs are primarily intended for managing data access within the Databricks workspace for BI and data science use cases. For sensitive data engineering data, you will want to make sure that you limit access to the storage containers using your cloud identity management.

In [0]:
try:
    spark.sql("SHOW GRANT ON TABLE bronze")
    
except py4j.protocol.Py4JJavaError as e:
    print("Error: " + e.java_exception.getMessage())
    print("Solution: Consider enabling Table Access Control to demonstrate this feature.")

Error: Trying to perform permission action on Hive Metastore /CATALOG/`hive_metastore`/DATABASE/`dbacademy_odl_user_771624_databrickslabs_com_adewd_5_1`/TABLE/`bronze` but Table Access Control is not enabled on this cluster. 
Solution: Consider enabling Table Access Control to demonstrate this feature.

Run the following cell to delete the tables and files associated with this lesson.

In [0]:
DA.cleanup()

Dropping the database "dbacademy_odl_user_771624_databrickslabs_com_adewd_5_1"
Removing the working directory "dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/5.1"
Validating datasets...(0 seconds)

-sandbox
&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>